### Kansas Sorghum Experiments Data Cleaning Notebook
#### Data from Kansas State University Sorghum Experiments
- goal: to gather more cultivar data in addition to MAC Sorghum Seasons 4 & 6
- please contact Emily Cain at ejcain@arizona.edu with any questions or feedback

In [1]:
import datetime
import numpy as np
import pandas as pd

#### Read in data queried from betydb in `R` using this code:
```
library(traits)

options(betydb_url = "https://terraref.ncsa.illinois.edu/bety/",
        betydb_api_version = 'v1',
        betydb_key = 'secret_api_key_123456_abcde')
        
kansas <- betydb_query(experiment  = "~KSU",
                         limit     =  "none")
                      
write.csv(kansas, file = "kansas_experiments_2020-03-24.csv")
```

In [3]:
df_0 = pd.read_csv('ksu_experiments_2020-03-24.csv', low_memory=False)
print(df_0.shape)
df_0.head(3)

(1552266, 39)


,Unnamed: 0,checked,result_type,id,citation_id,site_id,treatment_id,sitename,city,lat,...,n,statname,stat,notes,access_level,cultivar,entity,method_name,view_url,edit_url
0,1,0,traits,6004546896,6.000000e+09,6000005848,6.000000e+09,MAC Field Scanner Season 4 Range 38 Column 14,Maricopa,33.075878,...,NaN,NaN,NaN,NaN,2,PI218112,NaN,3D scanner to leaf angle distribution,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
1,2,0,traits,6004555749,6.000000e+09,6000005848,6.000000e+09,MAC Field Scanner Season 4 Range 38 Column 14,Maricopa,33.075878,...,NaN,NaN,NaN,NaN,2,PI218112,NaN,3D scanner to leaf angle distribution,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...
2,3,0,traits,6004555750,6.000000e+09,6000005848,6.000000e+09,MAC Field Scanner Season 4 Range 38 Column 14,Maricopa,33.075878,...,NaN,NaN,NaN,NaN,2,PI218112,NaN,3D scanner to leaf angle distribution,https://terraref.ncsa.illinois.edu/bety/traits...,https://terraref.ncsa.illinois.edu/bety/traits...


#### Find sitenames that do **not** start with `MAC`
- Slice dataframe to only include sitenames that include `KSU`

In [4]:
non_mac_sites = df_0[~df_0.sitename.str.startswith('MAC')]
print(non_mac_sites.shape)
# non_mac_sites.head(3)

(41540, 39)


In [5]:
print(non_mac_sites.raw_date.min())
print(non_mac_sites.raw_date.max())

2014-06-03 13:11:14 -0500
2016-10-21 00:00:00 -0500


In [6]:
ksu_0 = non_mac_sites[non_mac_sites.sitename.str.contains('KSU')]
print(ksu_0.shape)
# ksu_0.tail(3)

(37353, 39)


#### Slice for selected traits
- canopy height
- days & GDD to flowering
- may use other traits as needed for future models

In [7]:
ksu_0.trait.unique()

array(['canopy_height', 'lodging_percent', 'emergence_score',
       'seedling_vigor', 'Sugar_content', 'crude_protein', 'stem_width',
       'flowering_time', 'crown_color',
       'aboveground_fresh_biomass_per_plot', 'leaf_length', 'leaf_width',
       'leaf_attachment_angle', 'adf', 'ndf',
       'aboveground_biomass_moisture'], dtype=object)

In [8]:
ksu_1 = ksu_0.loc[(ksu_0.trait == 'flowering_time') | (ksu_0.trait == 'canopy_height')]
print(ksu_1.shape)
# ksu_1.head(3)

(5497, 39)


#### Drop & Rename Columns
- rename `mean` to `value`
- convert `raw_date` to new datetime object
- new datetime object will be in `date` column

In [9]:
# ksu_1.columns

In [10]:
# Can drop most columns with only one value

# for col in ksu_1.columns:
    
#     if ksu_1[col].nunique() < 5:
#         print(f'Unique values for {col}: {ksu_1[col].unique()}')

In [11]:
cols_to_drop = ['Unnamed: 0', 'checked', 'result_type', 'id', 'citation_id', 'site_id', 'treatment_id', 
                'city', 'scientificname', 'commonname', 'genus', 'species_id', 'cultivar_id', 'author', 
                'citation_year', 'time', 'month', 'year', 'n', 'statname', 'stat', 'notes', 'access_level', 
                'entity', 'view_url', 'edit_url', 'treatment', 'date', 'dateloc']

ksu_2 = ksu_1.drop(labels=cols_to_drop, axis=1)
print(ksu_2.shape)
# ksu_2.tail(3)

(5497, 10)


#### Convert `raw_date` to datetime object

In [12]:
ksu_2.dtypes

sitename              object
lat                  float64
lon                  float64
raw_date              object
trait                 object
trait_description     object
mean                 float64
units                 object
cultivar              object
method_name           object
dtype: object

In [13]:
new_dates = pd.to_datetime(ksu_2.raw_date)

ksu_3 = ksu_2.copy()
ksu_3['date'] = new_dates

print(ksu_2.shape[0])
print(ksu_3.shape[0])

# ksu_3.head(3)

5497
5497


In [14]:
ksu_4 = ksu_3.rename({'mean': 'value'}, axis=1)
print(ksu_4.shape)
# ksu_4.tail(3)

(5497, 11)


### Extract `Range` and `Pass` values
- still need to determine how the field is structured
- is `Pass` similar to `Column` in the MAC experiments?

In [15]:
ksu_5 = ksu_4.copy()

ksu_5['range'] = ksu_5['sitename'].str.extract("Range (\d+)").astype(int)
ksu_5['pass'] = ksu_5['sitename'].str.extract("Pass (\d+)").astype(int)

# ksu_5.sample(n=5)

### Growing Degree Days (GDD) to Flowering
- Weather data taken from [KSU Weather Station](http://mesonet.k-state.edu/weather/historical/) in Manhattan
- planting date: 2016-06-17
- harvest date: 2016-10-21

In [16]:
manhattan_weather_0 = pd.read_csv('manhattan_weather_2016_daily.csv')
print(manhattan_weather_0.shape)
manhattan_weather_0.head(5)

(155, 15)


,Timestamp,Station,AirTemperature,AirTemperature.1,RelativeHumidity,Precipitation,WindSpeed2m,WindSpeed2m.1,SoilTemperature5cm,SoilTemperature5cm.1,SoilTemperature10cm,SoilTemperature10cm.1,SolarRadiation,ETo,ETo.1
0,NaN,NaN,max,min,avg,total,avg,max,max,min,max,min,total,grass,alfalfa
1,NaN,NaN,°F,°F,%,inches,mph,mph,°F,°F,°F,°F,ly,inches,inches
2,2016-06-01,Manhattan,81.7,57.9,65.9,0,3.1,13.8,82.1,66.7,78.4,68.1,660.1,0.21,0.25
3,2016-06-02,Manhattan,84.8,56.1,59.6,0,2.8,14,85.1,66.8,80.7,68.4,673.7,0.22,0.26
4,2016-06-03,Manhattan,85.8,55.4,63.2,0.15,3.9,22.9,85.8,67.9,81.1,69.5,667.8,0.23,0.28


#### Change column names and drop first two rows
- Add datetime column

In [17]:
manhattan_weather_1 = manhattan_weather_0.copy()

datetimes = pd.to_datetime(manhattan_weather_1['Timestamp'])
manhattan_weather_1['date'] = datetimes

print(manhattan_weather_1.shape)
# manhattan_weather_1.tail()

(155, 16)


In [18]:
# manhattan_weather_1.columns

In [19]:
# Drop first 2 rows

manhattan_weather_2 = manhattan_weather_1.iloc[2:]
print(manhattan_weather_2.shape)
# manhattan_weather_2.head()

(153, 16)


In [20]:
# Drop `timestamp` column

manhattan_weather_3 = manhattan_weather_2.drop(labels=['Timestamp'], axis=1)
print(manhattan_weather_3.shape)
# manhattan_weather_3.head()

(153, 15)


In [21]:
manhattan_weather_4 = manhattan_weather_3.rename({'Station': 'station', 'AirTemperature': 'air_temp_max_F', 
                                                  'AirTemperature.1': 'air_temp_min_F', 'RelativeHumidity': 'avg_rh',
                                                  'Precipitation': 'precip_total', 'WindSpeed2m': 'avg_wind_speed', 
                                                  'WindSpeed2m.1': 'max_wind_speed', 'SoilTemperature5cm': 'soil_temp_5cm_max',
                                                  'SoilTemperature5cm.1': 'soil_temp_5cm_min', 
                                                  'SoilTemperature10cm': 'soil_temp_10cm_max', 
                                                  'SoilTemperature10cm.1': 'soil_temp_10cm_min', 'SolarRadiation': 'solar_rad',
                                                  'ETo': 'eto_grass', 'ETo.1': 'eto_alfalfa'}, axis=1)
print(manhattan_weather_4.shape)
# manhattan_weather_4.head()

(153, 15)


#### Add Day-of-year (DOY) to Weather Dataframe
- slice dataframe to only include season dates from planting to harvest
- change `date` to index, but keep `date` column
- use Pandas `PeriodIndex.dayofyear()`

In [22]:
manhattan_weather_5 = manhattan_weather_4.loc[(manhattan_weather_4['date'] >= '2016-06-17') & (manhattan_weather_4['date'] <= '2016-10-21')]

In [23]:
manhattan_weather_6 = manhattan_weather_5.set_index(keys=['date'], drop=False)
print(manhattan_weather_6.shape)
# manhattan_weather_6.tail(3)

(127, 15)


In [24]:
manhattan_weather_7 = manhattan_weather_6.copy()

manhattan_weather_7['day_of_year'] = manhattan_weather_7.index.dayofyear

In [25]:
# manhattan_weather_7.tail(3)

#### Add Growing Degree Days (GDD)
- convert all numeric columns from string `to_numeric`
- add air temps in C
- equation = (F - 32) x 0.5556 = C
- daily gdd equation = ((max_air_temp + min_air_temp) / 2) - 10

In [26]:
cols_to_convert = ['air_temp_max_F', 'air_temp_min_F', 'avg_rh', 'precip_total', 'avg_wind_speed', 'max_wind_speed', 
                   'soil_temp_5cm_max', 'soil_temp_5cm_min', 'soil_temp_10cm_max', 'soil_temp_10cm_min', 'solar_rad', 
                   'eto_grass', 'eto_alfalfa']

In [27]:
manhattan_weather_7[cols_to_convert] = manhattan_weather_7[cols_to_convert].apply(pd.to_numeric)

In [28]:
manhattan_weather_7.dtypes

station                       object
air_temp_max_F               float64
air_temp_min_F               float64
avg_rh                       float64
precip_total                 float64
avg_wind_speed               float64
max_wind_speed               float64
soil_temp_5cm_max            float64
soil_temp_5cm_min            float64
soil_temp_10cm_max           float64
soil_temp_10cm_min           float64
solar_rad                    float64
eto_grass                    float64
eto_alfalfa                  float64
date                  datetime64[ns]
day_of_year                    int64
dtype: object

In [29]:
manhattan_weather_8 = manhattan_weather_7.copy()

manhattan_weather_8['air_temp_max_C'] = round(((manhattan_weather_8['air_temp_max_F'] - 32) * 0.556), 1)
print(manhattan_weather_8.shape)
# manhattan_weather_8.tail(3)

(127, 17)


In [30]:
manhattan_weather_9 = manhattan_weather_8.copy()

manhattan_weather_9['air_temp_min_C'] = round(((manhattan_weather_9['air_temp_min_F'] - 32) * 0.556), 1)
print(manhattan_weather_9.shape)
# manhattan_weather_9.head(3)

(127, 18)


In [31]:
manhattan_weather_10 = manhattan_weather_9.copy()

manhattan_weather_10['daily_gdd'] = (((manhattan_weather_10['air_temp_max_C'] + manhattan_weather_10['air_temp_min_C'])) / 2) - 10

print(manhattan_weather_10.shape)
# manhattan_weather_10.sample(n=3)

(127, 19)


In [32]:
# Check for any negative daily GDD values (if any, need to be converted to 0)

manhattan_weather_10.loc[manhattan_weather_10.daily_gdd < 0]

,station,air_temp_max_F,air_temp_min_F,avg_rh,precip_total,avg_wind_speed,max_wind_speed,soil_temp_5cm_max,soil_temp_5cm_min,soil_temp_10cm_max,soil_temp_10cm_min,solar_rad,eto_grass,eto_alfalfa,date,day_of_year,air_temp_max_C,air_temp_min_C,daily_gdd
date,,,,,,,,,,,,,,,,,,,
2016-10-12,Manhattan,58.2,40.3,70.9,0.0,7.7,19.6,63.0,57.2,63.9,59.2,332.7,0.09,0.14,2016-10-12,286,14.6,4.6,-0.40
2016-10-13,Manhattan,60.3,33.4,72.3,0.0,2.8,11.7,60.0,53.9,60.2,56.2,304.5,0.07,0.10,2016-10-13,287,15.7,0.8,-1.75


In [33]:
# Assign negative daily gdd values to 0

manhattan_weather_11 = manhattan_weather_10.copy()

In [34]:
# ignore SeetingWithCopyWarning

manhattan_weather_11['daily_gdd']['2016-10-12'] = 0
manhattan_weather_11['daily_gdd']['2016-10-13'] = 0

/anaconda3/envs/ua-ag/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/envs/ua-ag/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [35]:
# Check to see that negative values were successfully converted to 0

manhattan_weather_11.loc[manhattan_weather_11.daily_gdd <= 0]

,station,air_temp_max_F,air_temp_min_F,avg_rh,precip_total,avg_wind_speed,max_wind_speed,soil_temp_5cm_max,soil_temp_5cm_min,soil_temp_10cm_max,soil_temp_10cm_min,solar_rad,eto_grass,eto_alfalfa,date,day_of_year,air_temp_max_C,air_temp_min_C,daily_gdd
date,,,,,,,,,,,,,,,,,,,
2016-10-12,Manhattan,58.2,40.3,70.9,0.0,7.7,19.6,63.0,57.2,63.9,59.2,332.7,0.09,0.14,2016-10-12,286,14.6,4.6,0.0
2016-10-13,Manhattan,60.3,33.4,72.3,0.0,2.8,11.7,60.0,53.9,60.2,56.2,304.5,0.07,0.10,2016-10-13,287,15.7,0.8,0.0


In [36]:
# should now return an empty df

manhattan_weather_11.loc[manhattan_weather_11.daily_gdd < 0]

,station,air_temp_max_F,air_temp_min_F,avg_rh,precip_total,avg_wind_speed,max_wind_speed,soil_temp_5cm_max,soil_temp_5cm_min,soil_temp_10cm_max,soil_temp_10cm_min,solar_rad,eto_grass,eto_alfalfa,date,day_of_year,air_temp_max_C,air_temp_min_C,daily_gdd
date,,,,,,,,,,,,,,,,,,,


In [37]:
# Add cumulative GDD, round to nearest integer

manhattan_weather_12 = manhattan_weather_11.copy()

manhattan_weather_12['gdd'] = np.rint(np.cumsum(manhattan_weather_12['daily_gdd']))
print(manhattan_weather_12.shape)
# manhattan_weather_12.tail()

(127, 20)


Drop `daily_gdd`

In [38]:
manhattan_weather_13 = manhattan_weather_12.drop(labels=['daily_gdd'], axis=1)
print(manhattan_weather_13.shape)
# manhattan_weather_13.head()

(127, 19)


#### Write Manhattan Weather Data to `.csv`

In [39]:
timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
output_filename = f'ksu_weather_2016_daily_{timestamp}.csv'.replace(':', '')

manhattan_weather_13.to_csv(output_filename)

#### Add Day of Year & GDD to Days to Flowering DataFrame
- slice trait data to only include `days_to_flowering`
- merge DataFrames on `date_of_flowering`

In [40]:
ksu_5.trait.unique()

array(['canopy_height', 'flowering_time'], dtype=object)

In [41]:
flowering_df_0 = ksu_5.loc[ksu_5.trait == 'flowering_time']
print(flowering_df_0.shape)
# flowering_df_0.head(3)

(224, 13)


In [42]:
flowering_df_0.cultivar.nunique()

128

#### Add `planting_date`
- 2016-06-17

In [43]:
day_of_planting = datetime.date(2016,6,17)
flowering_df_1 = flowering_df_0.copy()

flowering_df_1['date_of_planting'] = day_of_planting
print(flowering_df_1.shape)
# flowering_df_1.head(3)

(224, 14)


#### Create timedelta using `flowering_time` values

In [44]:
timedelta_values = flowering_df_1['value'].values
dates_of_flowering = []

for val in timedelta_values:
    
    date_of_flowering = day_of_planting + datetime.timedelta(days=val)
    dates_of_flowering.append(date_of_flowering)
    
print(flowering_df_1.shape[0])
print(len(dates_of_flowering))

224
224


In [45]:
flowering_df_2 = flowering_df_1.copy()
flowering_df_2['date_of_flowering'] = dates_of_flowering
print(flowering_df_2.shape)
# flowering_df_2.sample(n = 3)

(224, 15)


#### Add GDD and day_of_year to flowering DataFrame

In [46]:
ksu_gdd = manhattan_weather_13[['date', 'day_of_year', 'gdd']]

In [47]:
ksu_gdd.head()

,date,day_of_year,gdd
date,,,
2016-06-17,2016-06-17,169,21.0
2016-06-18,2016-06-18,170,38.0
2016-06-19,2016-06-19,171,56.0
2016-06-20,2016-06-20,172,77.0
2016-06-21,2016-06-21,173,97.0


In [48]:
flowering_df_3 = flowering_df_2.copy()
flowering_df_3.date_of_flowering = pd.to_datetime(flowering_df_3.date_of_flowering)
flowering_df_3.dtypes

sitename                                             object
lat                                                 float64
lon                                                 float64
raw_date                                             object
trait                                                object
trait_description                                    object
value                                               float64
units                                                object
cultivar                                             object
method_name                                          object
date                 datetime64[ns, pytz.FixedOffset(-300)]
range                                                 int64
pass                                                  int64
date_of_planting                                     object
date_of_flowering                            datetime64[ns]
dtype: object

In [49]:
flowering_df_4 = flowering_df_3.merge(ksu_gdd, how='left', left_on='date_of_flowering', right_on=ksu_gdd.index)
print(flowering_df_4.shape)
# flowering_df_4.head(3)

(224, 18)


#### Drop all date columns except `date_of_flowering`

In [50]:
date_cols_to_drop = ['date_x', 'raw_date', 'date_of_planting', 'date_y']
flowering_df_5 = flowering_df_4.drop(labels=date_cols_to_drop, axis=1)
print(flowering_df_5.shape)
# flowering_df_5.tail(3)

(224, 14)


#### Check for duplicates

In [51]:
flowering_df_5.duplicated().value_counts()

False    224
dtype: int64

#### Sort flowering dataframe by `date`

In [52]:
flowering_df_6 = flowering_df_5.sort_values(by='date_of_flowering', ascending=True).reset_index(drop=True)
# flowering_df_6.head()

#### Write flowering dataframe to `.csv`

In [53]:
timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
output_filename = f'ksu_flowering_{timestamp}.csv'.replace(':', '')

flowering_df_6.to_csv(output_filename, index=False)

#### Canopy Height DataFrame

In [54]:
ksu_5.trait.value_counts()

canopy_height     5273
flowering_time     224
Name: trait, dtype: int64

In [55]:
canopy_0 = ksu_5.loc[ksu_5.trait == 'canopy_height']
print(canopy_0.shape)
# canopy_0.head(3)

(5273, 13)


#### Drop `raw_date`

In [56]:
canopy_1 = canopy_0.drop(labels=['raw_date'], axis=1)
print(canopy_1.shape)
# canopy_1.head(3)

(5273, 12)


#### Sort by Date

In [57]:
canopy_2 = canopy_1.copy()

canopy_2['date'] = canopy_2['date'].astype('datetime64[ns]')
canopy_2.dtypes

sitename                     object
lat                         float64
lon                         float64
trait                        object
trait_description            object
value                       float64
units                        object
cultivar                     object
method_name                  object
date                 datetime64[ns]
range                         int64
pass                          int64
dtype: object

In [58]:
canopy_3 = canopy_2.set_index(keys=['date'], drop=True)
print(canopy_3.shape)
# canopy_3.head()

(5273, 11)


In [59]:
canopy_4 = canopy_3.sort_index()
print(canopy_4.shape)
# canopy_4.head()

(5273, 11)


#### Write to `.csv`

In [60]:
timestamp = datetime.datetime.now().replace(microsecond=0).isoformat()
output_filename = f'ksu_canopy_heights_{timestamp}.csv'.replace(':', '')

canopy_4.to_csv(output_filename, index=False)